In [1]:
import re

def clean_latex_string(s):
    # Remove leading and trailing $
    s = s.strip('$')
    # Remove all backslashes
    s = s.replace('\\', '')
    # Replace 'pm' (from '\pm' or any residue) with ±
    s = re.sub(r'\s*pm\s*', ' +- ', s)
    return s

def latex_to_results(latex_str):
    tabular_match = re.search(r'\\begin{tabular}{.*?}(.*?)\\end{tabular}', latex_str, re.DOTALL)
    if not tabular_match:
        raise ValueError("No tabular block found.")
    tabular_str = tabular_match.group(1)

    results = []
    for line in tabular_str.splitlines():
        line = line.strip()
        if (
            not line
            or line.startswith("\\")
            or '&' not in line
            or 'Pair' in line
            or 'Score' in line
        ):
            continue

        # Remove trailing \\
        if line.endswith('\\\\'):
            line = line[:-2].strip()
        elif line.endswith('\\'):
            line = line[:-1].strip()

        cols = [c.strip() for c in line.split('&')]
        if len(cols) != 7:
            continue  # Skip malformed lines

        # Parse columns
        pair = clean_latex_string(cols[0])
        cointegration_score_str = cols[1]

        sci_match = re.match(
            r'^\$?([-\d\.]+)\\times\s*10\^\{(-?\d+)\}\$?$', cointegration_score_str)
        if sci_match:
            cointegration_score = float(sci_match.group(1)) * (10 ** int(sci_match.group(2)))
        else:
            try:
                cointegration_score = float(cointegration_score_str.replace('$', '').replace('%', '').strip())
            except:
                cointegration_score = cointegration_score_str

        val_mse = float(cols[2])
        test_mse = float(cols[3])
        yoy_str = clean_latex_string(cols[4])
        gt_yoy = clean_latex_string(cols[5])
        returns_score = float(cols[6])

        results.append((pair, cointegration_score, val_mse, test_mse, yoy_str, gt_yoy, returns_score))
    return results


import re

def print_averages(results):
    results_clean = [r for r in results if r[-1] != -1 and r[-1] <= 1.0]  

    val_mse_list = []
    test_mse_list = []
    yoy_mean_list = []
    yoy_stddev_list = []
    gt_yoy_list = []
    returns_score_list = []

    for r in results_clean:
        _, _, val_mse, test_mse, yoy_str, gt_yoy_str, returns_score = r

        # Validation and test MSE
        val_mse_list.append(float(val_mse))
        test_mse_list.append(float(test_mse))

        # yoy_str, e.g., '3.11% +- 0.23%'
        yoy_match = re.match(r'([-0-9.]+)% \+\- ([0-9.]+)%', yoy_str)
        if yoy_match:
            yoy_mean = float(yoy_match.group(1))
            yoy_stddev = float(yoy_match.group(2))
            yoy_mean_list.append(yoy_mean)
            yoy_stddev_list.append(yoy_stddev)
        else:
            # Handle if not in expected format (fallback to nan)
            yoy_mean_list.append(float('nan'))
            yoy_stddev_list.append(float('nan'))

        # gt_yoy, e.g., '3.31%' or '-100.00%'
        try:
            gt_yoy = float(gt_yoy_str.replace('%', ''))
        except:
            gt_yoy = float('nan')
        gt_yoy_list.append(gt_yoy)

        # returns_score
        returns_score_list.append(float(returns_score))

    import numpy as np

    def safe_mean(lst):
        arr = np.array(lst)
        return f"{np.nanmean(arr):.5f}"
    
    def safe_std(lst):
        arr = np.array(lst)
        return f"{np.nanstd(arr):.5f}"

    def safe_median(lst):
        arr = np.array(lst)
        return f"{np.nanmedian(arr):.5f}"

    print("Average (std dev) val_mse:", safe_mean(val_mse_list), "+-", safe_std(val_mse_list))
    print("Average (std dev) test_mse:", safe_mean(test_mse_list), "+-", safe_std(test_mse_list))
    print("Average yoy mean:", safe_mean(yoy_mean_list))
    print("Average yoy stddev:", safe_mean(yoy_stddev_list))
    print("Average gt_yoy:", safe_mean(gt_yoy_list))
    print("Average (std dev) returns_score:", safe_mean(returns_score_list), "+-", safe_std(returns_score_list))
    
    print("Median val_mse:", safe_median(val_mse_list))
    print("Median test_mse:", safe_median(test_mse_list))
    print("Median yoy mean:", safe_median(yoy_mean_list))
    print("Median yoy stddev:", safe_median(yoy_stddev_list))
    print("Median gt_yoy:", safe_median(gt_yoy_list))
    print("Median returns_score:", safe_median(returns_score_list))

    
    

In [ ]:
import numpy as np

kalman_results_strings = [r"""
\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule

1. (SHV,SMH) & $2.46\times 10^{-4}$ & 5.30985 & 7.83935 & $4.65\% \pm 0.33\%$ & TLOE* & nan \\
2. (SHV,ONEQ) & $4.04\times 10^{-4}$ & 19.79646 & 10.18280 & $2.85\% \pm 0.14\%$ & 33.16\% & 0.77 \\
3. (SHV,PHO) & $4.13\times 10^{-4}$ & 18.61065 & 10.64926 & $0.63\% \pm 0.04\%$ & 14.67\% & 0.88 \\
4. (SHV,PDP) & $9.15\times 10^{-4}$ & 16.39409 & 9.24491 & $3.05\% \pm 0.34\%$ & 43.99\% & 0.72 \\
5. (DVY,PEY) & $1.41\times 10^{-3}$ & 14.08359 & 23.33083 & $0.04\% \pm 0.02\%$ & 0.10\% & 1.00 \\
6. (PFF,EMB) & $1.45\times 10^{-3}$ & 45.18233 & 14.91943 & $0.93\% \pm 0.30\%$ & 0.71\% & 1.00 \\
7. (IGSB,BND) & $1.56\times 10^{-3}$ & 5.08239 & 41.93097 & $1.06\% \pm 0.12\%$ & -0.36\% & 1.01 \\
8. (IFGL,SHV) & $3.95\times 10^{-3}$ & 1.40722 & 33.98232 & TLOE* & 0.41\% & -1 \\
9. (PRFZ,SCZ) & $4.47\times 10^{-3}$ & 55.09569 & 5.65478 & $0.53\% \pm 0.14\%$ & 0.31\% & 1.00 \\
10. (IFGL,EMB) & $4.61\times 10^{-3}$ & 3.56071 & 25.63661 & $1.78\% \pm 0.25\%$ & 0.53\% & 1.01 \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}
""",

r"""
\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule
11. (PFF,EMB) & $7.71\times 10^{-4}$ & 15.04393 & 3.59531 & $0.47\% \pm 0.04\%$ & 0.35\% & 1.00 \\
12. (IFGL,EMB) & $1.98\times 10^{-3}$ & 13.77034 & 1.98746 & $0.68\% \pm 0.10\%$ & -0.03\% & 1.01 \\
13. (IFGL,SHV) & $2.30\times 10^{-3}$ & 6.25251 & 1.90164 & $1.24\% \pm 0.10\%$ & 0.32\% & 1.01 \\
14. (IGSB,BND) & $2.75\times 10^{-3}$ & 38.14638 & 9.04679 & $0.29\% \pm 0.06\%$ & 0.19\% & 1.00 \\
15. (IFGL,SOXX) & $2.88\times 10^{-3}$ & 9.54111 & 40.41774 & $3.09\% \pm 0.62\%$ & 23.38\% & 0.84 \\
16. (IFGL,SMH) & $2.91\times 10^{-3}$ & 9.08964 & 42.10597 & $2.86\% \pm 0.78\%$ & 19.67\% & 0.86 \\
17. (IFGL,PHO) & $2.93\times 10^{-3}$ & 24.46739 & 65.96253 & $2.05\% \pm 0.18\%$ & 12.06\% & 0.91 \\
18. (IFGL,PDP) & $2.97\times 10^{-3}$ & 15.33126 & 99.62816 & $0.86\% \pm 0.06\%$ & TLOE* & nan \\
19. (IFGL,FTCS) & $2.99\times 10^{-3}$ & 34.35278 & 72.52196 & $2.22\% \pm 0.19\%$ & 12.47\% & 0.91 \\
20. (IFGL,USIG) & $3.01\times 10^{-3}$ & 21.64105 & 7.28092 & $0.14\% \pm 0.08\%$ & -0.11\% & 1.00 \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}
""",
r"""

\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule
21. (PFF,EMB) & $5.58\times 10^{-4}$ & 1.96566 & 118.88822 & $5.35\% \pm 0.81\%$ & 0.61\% & 1.05 \\
22. (IFGL,EMB) & $1.40\times 10^{-3}$ & 4.31595 & 13.81962 & $1.18\% \pm 0.37\%$ & 0.43\% & 1.01 \\
23. (IGSB,BND) & $1.89\times 10^{-3}$ & 21.34905 & 129.56108 & $6.64\% \pm 0.36\%$ & 0.88\% & 1.06 \\
24. (USIG,IEI) & $2.78\times 10^{-3}$ & 1.98255 & 6.53487 & $2.12\% \pm 0.60\%$ & 1.30\% & 1.01 \\
25. (IGF,DVY) & $2.82\times 10^{-3}$ & 34.95064 & 184.19907 & $0.73\% \pm 0.10\%$ & TLOE* & nan \\
26. (DVY,PEY) & $3.45\times 10^{-3}$ & 1.18905 & 3.19544 & $0.17\% \pm 0.03\%$ & 0.27\% & 1.00 \\
27. (IGIB,IEI) & $4.50\times 10^{-3}$ & 3.87317 & 20.27943 & $3.19\% \pm 0.79\%$ & 1.15\% & 1.02 \\
28. (IFGL,SOXX) & $4.65\times 10^{-3}$ & 40.53388 & 31.40019 & $0.47\% \pm 0.07\%$ & TLOE* & nan \\
29. (IFGL,SMH) & $4.65\times 10^{-3}$ & 42.69896 & 36.01471 & $0.44\% \pm 0.07\%$ & TLOE* & nan \\
30. (IFGL,PHO) & $5.30\times 10^{-3}$ & 64.33764 & 59.88985 & $0.33\% \pm 0.08\%$ & 3.70\% & 0.97 \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}

""",
r"""
\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule
31. (PFF,EMB) & $2.82\times 10^{-4}$ & 25.90302 & 4.19535 & $0.13\% \pm 0.02\%$ & -0.06\% & 1.00 \\
32. (IFGL,EMB) & $7.70\times 10^{-4}$ & 1.36618 & 15.84792 & $0.24\% \pm 0.04\%$ & 0.65\% & 1.00 \\
33. (IGF,DVY) & $1.17\times 10^{-3}$ & 194.79982 & 16.58814 & $0.74\% \pm 0.07\%$ & TLOE* & nan \\
34. (IGIB,IEI) & $1.25\times 10^{-3}$ & 1.50374 & 2.88738 & $0.64\% \pm 0.07\%$ & 0.50\% & 1.00 \\
35. (DVY,PEY) & $1.57\times 10^{-3}$ & 1.08565 & 2.57658 & $0.01\% \pm 0.01\%$ & 0.15\% & 1.00 \\
36. (USIG,IEI) & $1.67\times 10^{-3}$ & 1.71368 & 6.12692 & $0.66\% \pm 0.06\%$ & 0.61\% & 1.00 \\
37. (IFGL,BND) & $1.93\times 10^{-3}$ & 5.82614 & 67.47981 & $0.36\% \pm 0.05\%$ & 0.25\% & 1.00 \\
38. (IFGL,SMH) & $2.58\times 10^{-3}$ & 42.12796 & 22.37654 & $4.33\% \pm 0.42\%$ & 16.05\% & 0.90 \\
39. (IFGL,MBB) & $2.63\times 10^{-3}$ & 4.28195 & 70.14928 & $0.36\% \pm 0.04\%$ & -0.27\% & 1.01 \\
40. (IFGL,SOXX) & $2.70\times 10^{-3}$ & 37.57428 & 24.65001 & $2.95\% \pm 0.32\%$ & TLOE* & nan \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}

""",
r"""

\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule
41. (PFF,EMB) & $1.38\times 10^{-4}$ & 17.60224 & 21.84058 & $0.87\% \pm 0.27\%$ & 0.10\% & 1.01 \\
42. (IFGL,MBB) & $4.45\times 10^{-4}$ & 64.24084 & 41.03698 & $0.80\% \pm 0.11\%$ & TLOE* & nan \\
43. (IFGL,EMB) & $7.27\times 10^{-4}$ & 5.02341 & 76.31407 & $1.06\% \pm 0.18\%$ & 0.39\% & 1.01 \\
44. (IGIB,IEI) & $7.97\times 10^{-4}$ & 5.21963 & 9.67679 & $0.78\% \pm 0.08\%$ & 0.05\% & 1.01 \\
45. (IGF,DVY) & $8.40\times 10^{-4}$ & 28.27241 & 122.67020 & $2.15\% \pm 0.36\%$ & TLOE* & nan \\
46. (USIG,IEI) & $1.09\times 10^{-3}$ & 12.47778 & 44.75450 & $0.74\% \pm 0.11\%$ & -0.25\% & 1.01 \\
47. (IFGL,BND) & $1.20\times 10^{-3}$ & 62.60961 & 60.42905 & $0.54\% \pm 0.07\%$ & TLOE* & nan \\
48. (IFGL,SMH) & $1.64\times 10^{-3}$ & 11.48536 & 42.76372 & $11.14\% \pm 0.83\%$ & 26.74\% & 0.88 \\
49. (IFGL,SOXX) & $1.82\times 10^{-3}$ & 16.04055 & 73.43191 & $6.16\% \pm 0.69\%$ & 11.34\% & 0.95 \\
50. (IGF,PPH) & $2.29\times 10^{-3}$ & 17.62344 & 37.60692 & $2.49\% \pm 0.32\%$ & TLOE* & nan \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}

"""
]

transformer_results_strings = [
r"""

\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule

1. (SHV,SMH) & $2.46\times 10^{-4}$ & 0.33193 & 2.22423 & TLOE* & TLOE* & nan \\
2. (SHV,ONEQ) & $4.04\times 10^{-4}$ & 0.09506 & 1.27594 & TLOE* & 33.16\% & -1 \\
3. (SHV,PHO) & $4.13\times 10^{-4}$ & 1.50177 & 1.61198 & $-10.92\% \pm 46.61\%$ & 14.67\% & 0.78 \\
4. (SHV,PDP) & $9.15\times 10^{-4}$ & 0.06638 & 0.63493 & TLOE* & 43.99\% & -1 \\
5. (DVY,PEY) & $1.41\times 10^{-3}$ & 0.09973 & 0.16805 & $0.07\% \pm 0.01\%$ & 0.10\% & 1.00 \\
6. (PFF,EMB) & $1.45\times 10^{-3}$ & 0.11254 & 0.32286 & $0.32\% \pm 0.01\%$ & 0.71\% & 1.00 \\
7. (IGSB,BND) & $1.56\times 10^{-3}$ & 0.02444 & 0.98751 & $0.44\% \pm 0.02\%$ & -0.36\% & 1.01 \\
8. (IFGL,SHV) & $3.95\times 10^{-3}$ & 0.25714 & 0.13834 & $-0.17\% \pm 0.04\%$ & 0.41\% & 0.99 \\
9. (PRFZ,SCZ) & $4.47\times 10^{-3}$ & 0.18200 & 0.08079 & $0.19\% \pm 0.02\%$ & 0.31\% & 1.00 \\
10. (IFGL,EMB) & $4.61\times 10^{-3}$ & 0.07168 & 0.30447 & $0.22\% \pm 0.05\%$ & 0.53\% & 1.00 \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}


""",
r"""

\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule

1. (PFF,EMB) & $7.71\times 10^{-4}$ & 0.32923 & 0.04681 & $0.28\% \pm 0.02\%$ & 0.35\% & 1.00 \\
2. (IFGL,EMB) & $1.98\times 10^{-3}$ & 0.10846 & 0.07437 & $-0.08\% \pm 0.03\%$ & -0.03\% & 1.00 \\
3. (IFGL,SHV) & $2.30\times 10^{-3}$ & 0.10044 & 0.08526 & $0.24\% \pm 0.02\%$ & 0.32\% & 1.00 \\
4. (IGSB,BND) & $2.75\times 10^{-3}$ & 1.06050 & 0.07438 & $0.18\% \pm 0.02\%$ & 0.19\% & 1.00 \\
5. (IFGL,SOXX) & $2.88\times 10^{-3}$ & 1.31220 & 18.40974 & TLOE* & 23.38\% & -1 \\
6. (IFGL,SMH) & $2.91\times 10^{-3}$ & 1.15956 & 16.40023 & $3.48\% \pm 0.16\%$ & 19.67\% & 0.86 \\
7. (IFGL,PHO) & $2.93\times 10^{-3}$ & 0.83101 & 9.25424 & $7.40\% \pm 0.15\%$ & 12.06\% & 0.96 \\
8. (IFGL,PDP) & $2.97\times 10^{-3}$ & 1.03267 & 12.14490 & TLOE* & TLOE* & nan \\
9. (IFGL,FTCS) & $2.99\times 10^{-3}$ & 0.76389 & 4.48763 & $8.12\% \pm 0.08\%$ & 12.47\% & 0.96 \\
10. (IFGL,USIG) & $3.01\times 10^{-3}$ & 0.10245 & 0.14323 & $-21.66\% \pm 41.49\%$ & -0.11\% & 0.78 \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}

""",
r"""

\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule

1. (PFF,EMB) & $5.58\times 10^{-4}$ & 0.05525 & 0.07043 & $0.59\% \pm 0.06\%$ & 0.61\% & 1.00 \\
2. (IFGL,EMB) & $1.40\times 10^{-3}$ & 0.08056 & 0.04347 & $0.35\% \pm 0.03\%$ & 0.43\% & 1.00 \\
3. (IGSB,BND) & $1.89\times 10^{-3}$ & 0.08040 & 0.24700 & $0.83\% \pm 0.03\%$ & 0.88\% & 1.00 \\
4. (USIG,IEI) & $2.78\times 10^{-3}$ & 0.05681 & 0.04934 & $1.10\% \pm 0.05\%$ & 1.30\% & 1.00 \\
5. (IGF,DVY) & $2.82\times 10^{-3}$ & 1.85194 & 8.62643 & TLOE* & TLOE* & nan \\
6. (DVY,PEY) & $3.45\times 10^{-3}$ & 0.03509 & 0.11978 & $0.36\% \pm 0.02\%$ & 0.27\% & 1.00 \\
7. (IGIB,IEI) & $4.50\times 10^{-3}$ & 0.05859 & 0.02447 & $1.12\% \pm 0.02\%$ & 1.15\% & 1.00 \\
8. (IFGL,SOXX) & $4.65\times 10^{-3}$ & 0.79553 & 0.95163 & TLOE* & TLOE* & nan \\
9. (IFGL,SMH) & $4.65\times 10^{-3}$ & 0.63356 & 0.78920 & TLOE* & TLOE* & nan \\
10. (IFGL,PHO) & $5.30\times 10^{-3}$ & 1.01711 & 1.97333 & TLOE* & 3.70\% & -1 \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}

""",
r"""

\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule

1. (PFF,EMB) & $2.82\times 10^{-4}$ & 0.06810 & 0.16529 & $0.09\% \pm 0.01\%$ & -0.06\% & 1.00 \\
2. (IFGL,EMB) & $7.70\times 10^{-4}$ & 0.03479 & 0.09328 & $0.43\% \pm 0.07\%$ & 0.65\% & 1.00 \\
3. (IGF,DVY) & $1.17\times 10^{-3}$ & 0.15914 & 0.08200 & TLOE* & TLOE* & nan \\
4. (IGIB,IEI) & $1.25\times 10^{-3}$ & 0.02616 & 0.06271 & $0.48\% \pm 0.03\%$ & 0.50\% & 1.00 \\
5. (DVY,PEY) & $1.57\times 10^{-3}$ & 0.12656 & 0.05569 & $0.21\% \pm 0.01\%$ & 0.15\% & 1.00 \\
6. (USIG,IEI) & $1.67\times 10^{-3}$ & 0.04195 & 0.18978 & $0.35\% \pm 0.03\%$ & 0.61\% & 1.00 \\
7. (IFGL,BND) & $1.93\times 10^{-3}$ & 0.03225 & 0.14912 & $0.16\% \pm 0.09\%$ & 0.25\% & 1.00 \\
8. (IFGL,SMH) & $2.58\times 10^{-3}$ & 0.41073 & 0.48091 & $15.78\% \pm 0.31\%$ & 16.05\% & 1.00 \\
9. (IFGL,MBB) & $2.63\times 10^{-3}$ & 0.03245 & 0.22387 & $0.01\% \pm 0.08\%$ & -0.27\% & 1.00 \\
10. (IFGL,SOXX) & $2.70\times 10^{-3}$ & 0.41050 & 0.19171 & TLOE* & TLOE* & nan \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}

""", 
r"""
\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule

1. (PFF,EMB) & $1.38\times 10^{-4}$ & 0.15000 & 0.14245 & $0.11\% \pm 0.01\%$ & 0.10\% & 1.00 \\
2. (IFGL,MBB) & $4.45\times 10^{-4}$ & 0.47036 & 0.29027 & $-72.77\% \pm 47.72\%$ & TLOE* & nan \\
3. (IFGL,EMB) & $7.27\times 10^{-4}$ & 0.10872 & 0.15208 & $0.35\% \pm 0.02\%$ & 0.39\% & 1.00 \\
4. (IGIB,IEI) & $7.97\times 10^{-4}$ & 0.08394 & 0.09651 & $0.09\% \pm 0.00\%$ & 0.05\% & 1.00 \\
5. (IGF,DVY) & $8.40\times 10^{-4}$ & 0.07015 & 0.22685 & TLOE* & TLOE* & nan \\
6. (USIG,IEI) & $1.09\times 10^{-3}$ & 0.09081 & 0.25826 & $-0.21\% \pm 0.03\%$ & -0.25\% & 1.00 \\
7. (IFGL,BND) & $1.20\times 10^{-3}$ & 0.53958 & 0.44944 & TLOE* & TLOE* & nan \\
8. (IFGL,SMH) & $1.64\times 10^{-3}$ & 0.85777 & 17.19063 & $15.15\% \pm 0.46\%$ & 26.74\% & 0.91 \\
9. (IFGL,SOXX) & $1.82\times 10^{-3}$ & 0.27958 & 9.49619 & $15.51\% \pm 0.73\%$ & 11.34\% & 1.04 \\
10. (IGF,PPH) & $2.29\times 10^{-3}$ & 0.07948 & 0.41636 & TLOE* & TLOE* & nan \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}
"""
]

timemoe_results_strings = [
r"""
\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule

1. (SHV,SMH) & $2.46\times 10^{-4}$ & 0.02873 & 0.02923 & $-47.57\% \pm 69.90\%$ & TLOE* & nan \\
2. (SHV,ONEQ) & $4.04\times 10^{-4}$ & 0.05549 & 0.02700 & $-31.11\% \pm 59.98\%$ & 33.16\% & 0.52 \\
3. (SHV,PHO) & $4.13\times 10^{-4}$ & 0.03953 & 0.04419 & $-59.32\% \pm 55.14\%$ & 14.67\% & 0.35 \\
4. (SHV,PDP) & $9.15\times 10^{-4}$ & 0.03925 & 0.02771 & $-26.66\% \pm 63.59\%$ & 43.99\% & 0.51 \\
5. (DVY,PEY) & $1.41\times 10^{-3}$ & 0.08853 & 0.14001 & $0.04\% \pm 0.02\%$ & 0.10\% & 1.00 \\
6. (PFF,EMB) & $1.45\times 10^{-3}$ & 0.12620 & 0.22588 & $0.41\% \pm 0.15\%$ & 0.71\% & 1.00 \\
7. (IGSB,BND) & $1.56\times 10^{-3}$ & 0.01813 & 0.29892 & $0.06\% \pm 0.12\%$ & -0.36\% & 1.00 \\
8. (IFGL,SHV) & $3.95\times 10^{-3}$ & 0.29321 & 0.04271 & $0.13\% \pm 0.23\%$ & 0.41\% & 1.00 \\
9. (PRFZ,SCZ) & $4.47\times 10^{-3}$ & 0.22416 & 0.08381 & $0.25\% \pm 0.10\%$ & 0.31\% & 1.00 \\
10. (IFGL,EMB) & $4.61\times 10^{-3}$ & 0.08388 & 0.04807 & $0.81\% \pm 0.45\%$ & 0.53\% & 1.00 \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}
""",
r"""
\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule
11. (PFF,EMB) & $7.71\times 10^{-4}$ & 0.22252 & 0.05746 & $0.24\% \pm 0.08\%$ & 0.35\% & 1.00 \\
12. (IFGL,EMB) & $1.98\times 10^{-3}$ & 0.05223 & 0.08541 & $-0.15\% \pm 0.03\%$ & -0.03\% & 1.00 \\
13. (IFGL,SHV) & $2.30\times 10^{-3}$ & 0.04570 & 0.07188 & $0.26\% \pm 0.08\%$ & 0.32\% & 1.00 \\
14. (IGSB,BND) & $2.75\times 10^{-3}$ & 0.30943 & 0.09011 & $0.12\% \pm 0.05\%$ & 0.19\% & 1.00 \\
15. (IFGL,SOXX) & $2.88\times 10^{-3}$ & 0.02422 & 0.07939 & $-1.31\% \pm 51.50\%$ & 23.38\% & 0.80 \\
16. (IFGL,SMH) & $2.91\times 10^{-3}$ & 0.02260 & 0.08343 & $-44.97\% \pm 59.89\%$ & 19.67\% & 0.46 \\
17. (IFGL,PHO) & $2.93\times 10^{-3}$ & 0.03689 & 0.03839 & $9.21\% \pm 2.02\%$ & 12.06\% & 0.97 \\
18. (IFGL,PDP) & $2.97\times 10^{-3}$ & 0.02045 & 0.09252 & $-91.43\% \pm 32.19\%$ & TLOE* & nan \\
19. (IFGL,FTCS) & $2.99\times 10^{-3}$ & 0.04082 & 0.02703 & $9.89\% \pm 2.83\%$ & 12.47\% & 0.98 \\
20. (IFGL,USIG) & $3.01\times 10^{-3}$ & 0.05011 & 0.09892 & $-10.79\% \pm 31.19\%$ & -0.11\% & 0.89 \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}
""",
r"""

\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule
21. (PFF,EMB) & $5.58\times 10^{-4}$ & 0.06829 & 0.05012 & $0.54\% \pm 0.12\%$ & 0.61\% & 1.00 \\
22. (IFGL,EMB) & $1.40\times 10^{-3}$ & 0.09025 & 0.04573 & $0.33\% \pm 0.13\%$ & 0.43\% & 1.00 \\
23. (IGSB,BND) & $1.89\times 10^{-3}$ & 0.08831 & 0.01860 & $0.87\% \pm 0.09\%$ & 0.88\% & 1.00 \\
24. (USIG,IEI) & $2.78\times 10^{-3}$ & 0.05552 & 0.02754 & $0.89\% \pm 0.30\%$ & 1.30\% & 1.00 \\
25. (IGF,DVY) & $2.82\times 10^{-3}$ & 0.09323 & 0.20267 & TLOE* & TLOE* & nan \\
26. (DVY,PEY) & $3.45\times 10^{-3}$ & 0.03012 & 0.11510 & $0.07\% \pm 0.09\%$ & 0.27\% & 1.00 \\
27. (IGIB,IEI) & $4.50\times 10^{-3}$ & 0.06957 & 0.02443 & $0.83\% \pm 0.34\%$ & 1.15\% & 1.00 \\
28. (IFGL,SOXX) & $4.65\times 10^{-3}$ & 0.06592 & 0.17117 & TLOE* & TLOE* & nan \\
29. (IFGL,SMH) & $4.65\times 10^{-3}$ & 0.06931 & 0.19464 & TLOE* & TLOE* & nan \\
30. (IFGL,PHO) & $5.30\times 10^{-3}$ & 0.03054 & 0.05408 & $-61.78\% \pm 52.34\%$ & 3.70\% & 0.37 \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}
""",
r"""
\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule
31. (PFF,EMB) & $2.82\times 10^{-4}$ & 0.05109 & 0.09098 & $-0.02\% \pm 0.02\%$ & -0.06\% & 1.00 \\
32. (IFGL,EMB) & $7.70\times 10^{-4}$ & 0.04384 & 0.08899 & $0.53\% \pm 0.17\%$ & 0.65\% & 1.00 \\
33. (IGF,DVY) & $1.17\times 10^{-3}$ & 0.20808 & 0.05588 & TLOE* & TLOE* & nan \\
34. (IGIB,IEI) & $1.25\times 10^{-3}$ & 0.02433 & 0.06727 & $0.27\% \pm 0.19\%$ & 0.50\% & 1.00 \\
35. (DVY,PEY) & $1.57\times 10^{-3}$ & 0.11566 & 0.05366 & $0.12\% \pm 0.03\%$ & 0.15\% & 1.00 \\
36. (USIG,IEI) & $1.67\times 10^{-3}$ & 0.02768 & 0.08134 & $0.38\% \pm 0.24\%$ & 0.61\% & 1.00 \\
37. (IFGL,BND) & $1.93\times 10^{-3}$ & 0.03374 & 0.15046 & $0.46\% \pm 0.14\%$ & 0.25\% & 1.00 \\
38. (IFGL,SMH) & $2.58\times 10^{-3}$ & 0.25803 & 0.02390 & $-8.15\% \pm 48.13\%$ & 16.05\% & 0.79 \\
39. (IFGL,MBB) & $2.63\times 10^{-3}$ & 0.03106 & 0.18685 & $0.25\% \pm 0.10\%$ & -0.27\% & 1.01 \\
40. (IFGL,SOXX) & $2.70\times 10^{-3}$ & 0.22887 & 0.03220 & $-58.97\% \pm 55.89\%$ & TLOE* & nan \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}
""",
r"""

\begin{table}[h]
\centering
\small
\resizebox{\textwidth}{!}{
\begin{tabular}{lcccccc}
\toprule
Pair & Cointegration Score & val MSE & test MSE & YoY Returns (std) & \makecell{Theoretical Return\\Under Perfect\\Information} & Return Score \\
\midrule
41. (PFF,EMB) & $1.38\times 10^{-4}$ & 0.09239 & 0.09364 & $0.08\% \pm 0.02\%$ & 0.10\% & 1.00 \\
42. (IFGL,MBB) & $4.45\times 10^{-4}$ & 0.18489 & 0.09236 & $-42.19\% \pm 51.16\%$ & TLOE* & nan \\
43. (IFGL,EMB) & $7.27\times 10^{-4}$ & 0.08864 & 0.07025 & $0.44\% \pm 0.07\%$ & 0.39\% & 1.00 \\
44. (IGIB,IEI) & $7.97\times 10^{-4}$ & 0.06873 & 0.08799 & $0.02\% \pm 0.01\%$ & 0.05\% & 1.00 \\
45. (IGF,DVY) & $8.40\times 10^{-4}$ & 0.05464 & 0.16539 & TLOE* & TLOE* & nan \\
46. (USIG,IEI) & $1.09\times 10^{-3}$ & 0.08328 & 0.18845 & $-0.21\% \pm 0.05\%$ & -0.25\% & 1.00 \\
47. (IFGL,BND) & $1.20\times 10^{-3}$ & 0.14916 & 0.09422 & $-52.57\% \pm 52.18\%$ & TLOE* & nan \\
48. (IFGL,SMH) & $1.64\times 10^{-3}$ & 0.02001 & 0.14566 & $-21.83\% \pm 54.49\%$ & 26.74\% & 0.62 \\
49. (IFGL,SOXX) & $1.82\times 10^{-3}$ & 0.02896 & 0.16738 & $-7.09\% \pm 32.49\%$ & 11.34\% & 0.83 \\
50. (IGF,PPH) & $2.29\times 10^{-3}$ & 0.03039 & 0.03281 & TLOE* & TLOE* & nan \\
\bottomrule
\end{tabular}
}
\caption{Model performance and return statistics for all tested pairs.}
\end{table}
"""
]
kalman_results = [latex_to_results(string) for string in kalman_results_strings]
transformer_results = [latex_to_results(string) for string in transformer_results_strings]
timemoe_results = [latex_to_results(string) for string in timemoe_results_strings]

In [ ]:
yearly_kalman_val_mse = []
yearly_kalman_test_mse = []
yearly_kalman_return = []

yearly_transformer_val_mse = []
yearly_transformer_test_mse = []
yearly_transformer_return = []

yearly_timemoe_val_mse = []
yearly_timemoe_test_mse = []
yearly_timemoe_return = []

best_kalman_pair, best_kalman_test_mse = [], np.inf

for i, (kalman_result, transformer_result, timemoe_result) in enumerate(zip(kalman_results, transformer_results, timemoe_results)):
  # get stats for current year
   # ('1. (SHV,SMH)', 0.000246, 5.30985, 7.83935, '4.65% +- 0.33%', 'TLOE*', nan) 
   # tuple, return score, val mse, test mse, yoy_returns_str, return_score
  mean_kalman_val_mse = np.nanmean([t[2] for t in kalman_result])
  mean_kalman_test_mse = np.nanmean([t[3] for t in kalman_result])
  mean_kalman_return = np.nanmean([t[-1] for t in kalman_result])
  
  mean_transformer_val_mse = np.nanmean([t[2] for t in transformer_result])
  mean_transformer_test_mse = np.nanmean([t[3] for t in transformer_result])
  mean_transformer_return = np.nanmean([t[-1] for t in transformer_result])
  
  mean_timemoe_val_mse = np.nanmean([t[2] for t in timemoe_result])
  mean_timemoe_test_mse = np.nanmean([t[3] for t in timemoe_result])
  mean_timemoe_return = np.nanmean([t[-1] for t in timemoe_result])
  
  # Append to lists for cross-year averaging
  yearly_kalman_val_mse.append(mean_kalman_val_mse)
  yearly_kalman_test_mse.append(mean_kalman_test_mse)
  yearly_kalman_return.append(mean_kalman_return)

  yearly_transformer_val_mse.append(mean_transformer_val_mse)
  yearly_transformer_test_mse.append(mean_transformer_test_mse)
  yearly_transformer_return.append(mean_transformer_return)

  yearly_timemoe_val_mse.append(mean_timemoe_val_mse)
  yearly_timemoe_test_mse.append(mean_timemoe_test_mse)
  yearly_timemoe_return.append(mean_timemoe_return)
  
  # columns: kalman, transformer, timemoe
  current_str = f"""
\\hspace{{2em}}\\textit{{{2020 + i}}} & & & \\\\
\\hspace{{4em}}Validation MSE & ${mean_kalman_val_mse:.4f}$ & ${mean_transformer_val_mse:.4f}$ & ${mean_timemoe_val_mse:.4f}$ \\\\
\\hspace{{4em}}Test MSE & ${mean_kalman_test_mse:.4f}$ & ${mean_transformer_test_mse:.4f}$ & ${mean_timemoe_test_mse:.4f}$ \\\\
\\hspace{{4em}}Return Score & ${mean_kalman_return:.4f}$ & ${mean_transformer_return:.4f}$ & ${mean_timemoe_return:.4f}$ \\\\
  """
  print(current_str)
  
# Calculate overall averages across years
avg_yearly_kalman_val_mse = np.nanmean(yearly_kalman_val_mse)
avg_yearly_kalman_test_mse = np.nanmean(yearly_kalman_test_mse)
avg_yearly_kalman_return = np.nanmean(yearly_kalman_return)

avg_yearly_transformer_val_mse = np.nanmean(yearly_transformer_val_mse)
avg_yearly_transformer_test_mse = np.nanmean(yearly_transformer_test_mse)
avg_yearly_transformer_return = np.nanmean(yearly_transformer_return)

avg_yearly_timemoe_val_mse = np.nanmean(yearly_timemoe_val_mse)
avg_yearly_timemoe_test_mse = np.nanmean(yearly_timemoe_test_mse)
avg_yearly_timemoe_return = np.nanmean(yearly_timemoe_return)

# Example: Print overall averages in the same format
summary_str = f"""
\\hspace{{2em}}\\textit{{Average}} & & & \\\\
\\hspace{{4em}}\large{{Validation MSE}} & ${avg_yearly_kalman_val_mse:.4f}$ & ${avg_yearly_transformer_val_mse:.4f}$ & ${avg_yearly_timemoe_val_mse:.4f}$ \\\\
\\hspace{{4em}}\large{{Test MSE}} & ${avg_yearly_kalman_test_mse:.4f}$ & ${avg_yearly_transformer_test_mse:.4f}$ & ${avg_yearly_timemoe_test_mse:.4f}$ \\\\
\\hspace{{4em}}\large{{Return Score}} & ${avg_yearly_kalman_return:.4f}$ & ${avg_yearly_transformer_return:.4f}$ & ${avg_yearly_timemoe_return:.4f}$ \\\\
"""
print(summary_str)


\hspace{2em}\textit{2020} & & & \\
\hspace{4em}Validation MSE & $18.4523$ & $0.2743$ & $0.0997$ \\
\hspace{4em}Test MSE & $18.3371$ & $0.7749$ & $0.0968$ \\
\hspace{4em}Return Score & $0.7100$ & $0.5311$ & $0.8200$ \\
  

\hspace{2em}\textit{2021} & & & \\
\hspace{4em}Validation MSE & $18.7636$ & $0.6800$ & $0.0825$ \\
\hspace{4em}Test MSE & $34.4448$ & $6.1121$ & $0.0725$ \\
\hspace{4em}Return Score & $0.9489$ & $0.7289$ & $0.9000$ \\
  

\hspace{2em}\textit{2022} & & & \\
\hspace{4em}Validation MSE & $21.7197$ & $0.4665$ & $0.0661$ \\
\hspace{4em}Test MSE & $60.3782$ & $1.2895$ & $0.0904$ \\
\hspace{4em}Return Score & $1.0171$ & $0.7143$ & $0.9100$ \\
  

\hspace{2em}\textit{2023} & & & \\
\hspace{4em}Validation MSE & $31.6182$ & $0.1343$ & $0.1022$ \\
\hspace{4em}Test MSE & $23.2878$ & $0.1694$ & $0.0832$ \\
\hspace{4em}Return Score & $0.9888$ & $1.0000$ & $0.9750$ \\
  

\hspace{2em}\textit{2024} & & & \\
\hspace{4em}Validation MSE & $24.0595$ & $0.2730$ & $0.0801$ \\
\hspace{4em}